# Solving RBC Model with Dolo.jl

This worksheet demonstrates how to solve the RBC model with the [dolo](http://econforge.github.io/dolo/) library 
and how to generates impulse responses and stochastic simulations from the solution.

- This notebook is modified from `rbc_model` example distributed with dolo in : ``examples\notebooks\``.
- The model file(.yaml file) is in : ``examples\global_models\``


# YAML file for model specification

Dolo uses [YAML](http://www.yaml.org/spec/1.2/spec.html#Introduction) file which we can read locally or pull off the web. 

The YAML file works similarly to `.mod` file in Dynare, which does a configuration of the model. It includes the following:

1. In `symbols` section, specify different kinds of variables in the model:
    - `parameters:` deep parameters of the model, exp: $\beta$
    - `exogenous:` exogenous shocks of the model, exp: $\epsilon$ (iid shock on productivity)
    - `states:` state variables of the model, exp: $k$ (capital stock)
    - `controls:` control/choice variables of the model, exp: $n$ (labor)
    - `values:` value function in terms of Bellman Equation; `rewards:` utility function.

2. "auxiliary variables": these are variables that can be expressed as a closed form of the formerly-defined variables. Defining these variables will help to simplify the system of equations and sometimes these variables have particular economic meaning. These variables are defined in `definitions`. Once defined, one could use them in model equations.

3. Model section, specified using command `equations:`, consist of:

   Essential parts:
    
    - `arbitrage:` Euler Equation, notice the EE specification is usually accompanied by complementarity condition included in the `arbitrage` configuration.

    - `transition:` Law of motion, this should include both things like capital accumulation and process of shocks.

    Aditionals:

    - `expectations:` A function of purely expected future values.

    - `felicity:` defining utility function; `values:` defining value function;

    - `direct response:` additional equations that may speed up computation.


For more options in model specification please refer to http://www.econforge.org/Dolo.jl/latest/model_specification.html

# Example of RBC

In [1]:
using Dolo

In [2]:
filename="rbc_dtcc_iid.yaml"

"rbc_dtcc_iid.yaml"

In [3]:
;cat $filename

name: Real Business Cycle

model_type: dtcc

symbols:

   exogenous: [e_z]
   states: [z, k]
   controls: [n, i]
   expectations: [m]
   values: [V]
   parameters: [beta, sigma, eta, chi, delta, alpha, rho, zbar, sig_z]
   rewards: [u]

definitions:
    y: exp(z)*k^alpha*n^(1-alpha)
    c: y - i
    rk: alpha*y/k
    w: (1-alpha)*y/n

equations:

    arbitrage:
        - chi*n^eta*c^sigma - w                      | 0.0 <= n <= inf
        - 1 - beta*(c/c(1))^(sigma)*(1-delta+rk(1))  | 0.0 <= i <= inf


    transition:
        - z = rho*z(-1) + e_z
        - k = (1-delta)*k(-1) + i(-1)

    value:
        - V = c^(1-sigma)/(1-sigma) - chi*n^(1+eta)/(1+eta) + beta*V(1)

    felicity:
        - u =  c^(1-sigma)/(1-sigma) - chi*n^(1+eta)/(1+eta)

    expectation:
        - m = beta/c(1)^sigma*(1-delta+rk(1))

    direct_response:
        - n = ((1-alpha)*exp(z)*k^alpha*m/chi)^(1/(eta+alpha))
        - i = exp(z)*k^alpha*n^(1-alpha) - (m)^(-1/sigma)

calibration:

    # parameters
    beta 

`yaml_import(filename)` reads the YAML file and generates a model object. 

In [4]:
model = yaml_import(filename)

name,Real Business Cycle
filename,rbc_dtcc_iid.yaml
Type,Equation
expectation,\[m_{t} = \frac{\beta}{\left(c_{t+1}\right)^{\sigma}} \left(1-\delta\right)+rk_{t+1}\]
value,\[V_{t} = \left(\frac{\left(c_{t}\right)^{\left(1-\sigma\right)}}{\left(1-\sigma\right)}-\frac{\chi \left(n_{t}\right)^{1+\eta}}{1+\eta}\right)+\beta V_{t+1}\]
transition,"\[z_{t} = \rho z_{t-1}+e_{z,t}\]"
,\[k_{t} = \left(1-\delta\right) k_{t-1}+i_{t-1}\]
direct_response,\[n_{t} = \left(\frac{\left(1-\alpha\right) \text{exp}\left(z_{t}\right) \left(k_{t}\right)^{\alpha} m_{t}}{\chi}\right)^{\frac{1}{\eta+\alpha}}\]
,\[i_{t} = \left(\text{exp}\left(z_{t}\right) \left(k_{t}\right)^{\alpha} \left(n_{t}\right)^{\left(1-\alpha\right)}-\left(m_{t}\right)^{\frac{-1}{\sigma}}\right)\]
felicity,\[u_{t} = \left(\frac{\left(c_{t}\right)^{\left(1-\sigma\right)}}{\left(1-\sigma\right)}-\frac{\chi \left(n_{t}\right)^{1+\eta}}{1+\eta}\right)\]
arbitrage,\[\left(\chi \left(n_{t}\right)^{\eta} \left(c_{t}\right)^{\sigma}-w_{t}\right)\]


# Steady State values

The model file already has values for steady-state variables stated in the calibration section so we can go ahead and check that they are correct by computing the model equations at the steady state.

In [5]:
residuals(model)

Dict{Symbol,Array{Float64,1}} with 2 entries:
  :transition => [0.0, 0.0]
  :arbitrage  => [-4.44089e-16, 0.0]

Or more specifically one can check the s.s. values through command `model.calibration`

In [6]:
model.calibration[:controls]

2-element Array{Float64,1}:
 0.33    
 0.233874

# Solving the model

Dolo offers several [algorithms](http://www.econforge.org/Dolo.jl/latest/algos.html) to solve the model. Here we present a way of policy function iteration using command `time_iteration`, and its alternative faster command `improved_time_iteration`.

In [7]:
@time sol_global = time_iteration(model)

------------------------------------------------------------------
It    ϵₙ              ηₙ=|xₙ-xₙ₋₁|    λₙ=ηₙ/ηₙ₋₁      Time            Newton steps
------------------------------------------------------------------
1     1.19e+00        1.76e-01        NaN             2.08e+00        8    
2     1.28e-01        4.73e-02        2.69e-01        2.14e-02        5    
3     7.77e-02        3.50e-02        7.40e-01        2.72e-02        5    
4     5.21e-02        2.62e-02        7.49e-01        3.24e-02        5    
5     3.70e-02        1.99e-02        7.59e-01        1.57e-02        4    
6     2.72e-02        1.53e-02        7.69e-01        1.66e-02        4    
7     2.06e-02        1.19e-02        7.79e-01        1.77e-02        4    
8     1.59e-02        9.40e-03        7.89e-01        2.12e-02        4    
9     1.25e-02        7.50e-03        7.98e-01        2.54e-02        4    
10    9.94e-03        6.06e-03        8.08e-01        1.58e-02        4    
11    8.04e-03        4

Results of Time Iteration Algorithm
 * Complementarities: true
 * Discretized Process type: Dolo.DiscretizedIIDProcess
 * Decision Rule type: Dolo.CubicDR{Dolo.EmptyGrid,Dolo.CartesianGrid{2},2,2}
 * Number of iterations: 77
 * Convergence: true
   * |x - x'| < 1.0e-07: true


... and we get the decision rule:

In [8]:
dr_global = sol_global.dr

Dolo.CubicDR{Dolo.EmptyGrid,Dolo.CartesianGrid{2},2,2}


Or we could try the new improved time iteration algorithm

In [9]:
@time sol_global_fast = improved_time_iteration(model)

------------------------------------------------------------------------------------------------------------------------
N	f_x		d_x	Time_residuals	Time_inversion	Time_search	Lambda_0	N_invert	N_search	
------------------------------------------------------------------------------------------------------------------------
1      1.188337e+00 2.858994e-01     1.8778         2.1548         0.04057        0.000           60               3    
2      8.783762e-01 7.686131e-02     0.0040         0.0166         0.00126        0.000           72               1    
3      1.045269e-01 0.000000e+00     0.0127         0.0316         0.00137        0.000           136              1    
4      2.948941e-02 2.725911e-03     0.0056         0.0469         0.00103        0.000           98               1    
5      9.991518e-04 0.000000e+00     0.0046         0.0264         0.00099        0.000           98               1    
6      1.702487e-06 9.957221e-08     0.0044         0.0240         0.001

Results of Improved Time Iteration Algorithm
 * Number of iterations: 7
 * Complementarities: true
 * Decision Rule type: Dolo.CubicDR{Dolo.EmptyGrid,Dolo.CartesianGrid{2},2,2}
 * Convergence: true
 * Contractivity: 0.0
   * |x - x'| < 1.0e-08: true


# Decision rule(optional)

Here we plot optimal investment and labour for different levels of capital.

In [10]:
tab_global = Dolo.tabulate(model, dr_global, :k)

2-dimensional AxisArray{Float64,2,...} with axes:
    :V, Symbol[:e_z, :z, :k, :n, :i, :w, :rk, :y, :c]
    :k, [8.41948, 8.43838, 8.45728, 8.47618, 8.49508, 8.51398, 8.53287, 8.55177, 8.57067, 8.58957  …  10.1204, 10.1393, 10.1582, 10.1771, 10.196, 10.2149, 10.2338, 10.2527, 10.2716, 10.2905]
And data, a 9×100 Array{Float64,2}:
 0.0        0.0        0.0        …   0.0         0.0         0.0      
 0.0        0.0        0.0            0.0         0.0         0.0      
 8.41948    8.43838    8.45728       10.2527     10.2716     10.2905   
 0.34526    0.344932   0.344604       0.316985    0.316727    0.316469 
 0.243734   0.243535   0.243336       0.224283    0.224083    0.223883 
 1.92234    1.92437    1.92639    …   2.11012     2.11198     2.11383  
 0.0388267  0.0387438  0.0386611      0.0321328   0.0320757   0.0320187
 0.990608   0.990711   0.990812       0.998327    0.998388    0.998448 
 0.746874   0.747176   0.747476       0.774044    0.774305    0.774565 

In [11]:
Dolo.tabulate(model, dr_global,:k)[:n]

1-dimensional AxisArray{Float64,1,...} with axes:
    :k, [8.41948, 8.43838, 8.45728, 8.47618, 8.49508, 8.51398, 8.53287, 8.55177, 8.57067, 8.58957  …  10.1204, 10.1393, 10.1582, 10.1771, 10.196, 10.2149, 10.2338, 10.2527, 10.2716, 10.2905]
And data, a 100-element Array{Float64,1}:
 0.34526 
 0.344932
 0.344604
 0.344277
 0.34395 
 0.343623
 0.343297
 0.342971
 0.342646
 0.342323
 0.342   
 0.341678
 0.341358
 ⋮       
 0.319326
 0.319063
 0.318802
 0.318541
 0.31828 
 0.31802 
 0.317761
 0.317502
 0.317244
 0.316985
 0.316727
 0.316469

In [12]:
### Method for Pyplot
#using PyPlot

#figure(figsize=(8,3.5))

#subplot(121)
#plot(tab_global[:k], tab_global[:i], label="Global")
# plot(tab_pert['k'], tab_pert['i'], label='Perturbation')
#ylabel("i")
#title("Investment")
#legend()

#subplot(122)
#plot(tab_global[:k], tab_global[:n], label="Global")
## plot(tab_pert['k'], tab_pert['n'], label='Perturbation')
#ylabel("n")
#title("Labour")
#legend()

#tight_layout()
#show()

###

### Method for Plots.gr
using Plots
gr()

figure_1=Plots.plot(Plots.plot(tab_global[:k], tab_global[:i], label="Global", title = "Investment", ylabel = "i", line = 3),
Plots.plot(tab_global[:k], tab_global[:n], label="Global", title = "Labour", ylabel = "n", line =3))



<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 8.5 
 
 
 9.0 
 
 
 9.5 
 
 
 10.0 
 
 
 0.225 
 
 
 0.230 
 
 
 0.235 
 
 
 0.240 
 
 
 Investment 
 
 
 i 
 
<polyline clip-path="url(#clip9902)" style="stroke:#009af9; stroke-width:3; stroke-opacity:1; fill:none" points="
 61.1494,31.4961 63.3632,34.9484 65.577,38.4059 67.7908,41.8739 70.0046,45.3575 72.2184,48.8621 74.4322,52.3915 76.646,55.9404 78.8597,59.4991 81.0735,63.058 
 83.2873,66.6073 85.5011,70.1379 87.7149,73.6492 89.9287,77.1476 92.1425,80.64 94.3563,84.133 96.5701,87.6332 98.7839,91.1432 100.998,94.6599 103.211,98.1796 
 105.425,101.699 107.639,105.214 109.853,108.723 112.067,112.227 114.28,115.727 116.494,119.226 118.708,122.725 120.922,126.225 123.136,129.727 125.349,133.229 
 127.563,136.731 129.777,140.231 131.991,143.73 134.205,147.227 136.418,150.722 138.632,154.216 140.846,157.71 143.06,161.202 145.274,164.694 147.487,168.186 
 149.701,171.676 151.915,175.166 154.129,178.655 156.343,182.143 158.556,185.629 160.77,189.115 162.984,192.599 165.198,196.083 167.411,199.565 169.625,203.047 
 171.839,206.528 174.053,210.008 176.267,213.487 178.48,216.965 180.694,220.442 182.908,223.918 185.122,227.392 187.336,230.866 189.549,234.339 191.763,237.81 
 193.977,241.281 196.191,244.75 198.405,248.219 200.618,251.687 202.832,255.154 205.046,258.621 207.26,262.086 209.474,265.549 211.687,269.011 213.901,272.47 
 216.115,275.929 218.329,279.387 220.543,282.845 222.756,286.303 224.97,289.761 227.184,293.219 229.398,296.676 231.612,300.13 233.825,303.579 236.039,307.024 
 238.253,310.466 240.467,313.906 242.68,317.348 244.894,320.793 247.108,324.245 249.322,327.7 251.536,331.155 253.749,334.606 255.963,338.047 258.177,341.475 
 260.391,344.889 262.605,348.296 264.818,351.702 267.032,355.113 269.246,358.536 271.46,361.972 273.674,365.422 275.887,368.881 278.101,372.345 280.315,375.813 
 
 "/>
 
 
 
 
 Global 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 8.5 
 
 
 9.0 
 
 
 9.5 
 
 
 10.0 
 
 
 0.320 
 
 
 0.325 
 
 
 0.330 
 
 
 0.335 
 
 
 0.340 
 
 
 0.345 
 
 
 Labour 
 
 
 n 
 
<polyline clip-path="url(#clip9903)" style="stroke:#009af9; stroke-width:3; stroke-opacity:1; fill:none" points="
 361.149,31.4961 363.363,35.4157 365.577,39.3342 367.791,43.2503 370.005,47.1628 372.218,51.0706 374.432,54.9723 376.646,58.8664 378.86,62.7508 381.074,66.6237 
 383.287,70.4833 385.501,74.3278 387.715,78.1572 389.929,81.9733 392.143,85.7776 394.356,89.572 396.57,93.358 398.784,97.1364 400.998,100.906 403.211,104.667 
 405.425,108.417 407.639,112.157 409.853,115.885 412.067,119.601 414.28,123.307 416.494,127.003 418.708,130.689 420.922,134.366 423.136,138.034 425.349,141.692 
 427.563,145.34 429.777,148.979 431.991,152.608 434.205,156.227 436.418,159.836 438.632,163.435 440.846,167.025 443.06,170.606 445.274,174.178 447.487,177.74 
 449.701,181.293 451.915,184.836 454.129,188.371 456.343,191.896 458.556,195.412 460.77,198.918 462.984,202.416 465.198,205.905 467.411,209.385 469.625,212.855 
 471.839,216.317 474.053,219.77 476.267,223.215 478.48,226.65 480.694,230.077 482.908,233.495 485.122,236.904 487.336,240.305 489.549,243.697 491.763,247.08 
 493.977,250.455 496.191,253.822 498.405,257.18 500.618,260.53 502.832,263.871 505.046,267.204 507.26,270.529 509.474,273.846 511.687,277.154 513.901,280.453 
 516.115,283.745 518.329,287.028 520.543,290.303 522.756,293.571 524.97,296.831 527.184,300.084 529.398,303.328 531.612,306.564 533.825,309.791 536.039,313.01 
 538.253,316.22 540.467,319.423 542.68,322.618 544.894,325.806 547.108,328.989 549.322,332.165 551.536,335.334 553.749,338.495 555.963,341.646 558.177,344.787 
 560.391,347.918 562.605,351.039 564.818,354.153 567.032,357.259 569.246,360.36 571.46,363.457 573.674,366.549 575.887,369.639 578.101,372.727 580.315,375.813 
 
 "/>
 
 
 
 
 Global

Dolo also offers a convenient way to change parameter values in configuration. Use `set_calibration!(model,para=val)` command one can replace the original parameter value in the model file.

We can use this feature to do comparative stat easily. For example, let's consider a change in the value of $\delta$ in the model:

In [13]:
original_delta=model.calibration.flat[:delta] 

drs = []
delta_values = linspace(0.02, 0.05,5)
for val in delta_values
    print(val)
    set_calibration!(model, delta=val)
    push!(drs,time_iteration(model, verbose=false).dr)
end


0.020.02750.0350.04250.05

In [18]:
### Method for Pyplot
#figure(figsize=(5,3))

#for (i,dr) in enumerate(drs)
#     sim = Dolo.tabulate(model, dr,:k)
#     dv = delta_values[i]
#     plot(sim[:k],sim[:i], label="\$\\delta=$dv\$")
#end
#ylabel("i")
#title("Investment")
#legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
#show()

###

### Method for Plots.gr
figure_2 = Plots.plot()
for (i,dr) in enumerate(drs)
     sim = Dolo.tabulate(model, dr,:k)
     dv = delta_values[i]
     Plots.plot!(sim[:k],sim[:i], label="\\delta=$dv", ylabel = "i", title = "Investment", xlim = [3, 15], ylim = [0.19, 0.26], line = 3)
end

set_calibration!(model,delta=original_delta)

figure_2

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 3 
 
 
 6 
 
 
 9 
 
 
 12 
 
 
 15 
 
 
 0.19 
 
 
 0.20 
 
 
 0.21 
 
 
 0.22 
 
 
 0.23 
 
 
 0.24 
 
 
 0.25 
 
 
 0.26 
 
 
 Investment 
 
 
 i 
 
<polyline clip-path="url(#clip0302)" style="stroke:#009af9; stroke-width:3; stroke-opacity:1; fill:none" points="
 387.099,93.3637 388.14,94.5796 389.182,95.7973 390.223,97.0185 391.265,98.2448 392.306,99.478 393.348,100.719 394.389,101.967 395.431,103.218 396.472,104.469 
 397.514,105.716 398.555,106.956 399.597,108.19 400.638,109.419 401.68,110.646 402.721,111.873 403.763,113.103 404.804,114.336 405.846,115.571 406.887,116.807 
 407.929,118.042 408.97,119.276 410.011,120.508 411.053,121.738 412.094,122.966 413.136,124.193 414.177,125.421 415.219,126.649 416.26,127.877 417.302,129.106 
 418.343,130.334 419.385,131.561 420.426,132.788 421.468,134.014 422.509,135.239 423.551,136.464 424.592,137.688 425.634,138.912 426.675,140.135 427.717,141.359 
 428.758,142.582 429.8,143.804 430.841,145.026 431.882,146.247 432.924,147.468 433.965,148.688 435.007,149.908 436.048,151.128 437.09,152.347 438.131,153.565 
 439.173,154.783 440.214,156.001 441.256,157.218 442.297,158.435 443.339,159.651 444.38,160.867 445.422,162.082 446.463,163.297 447.505,164.511 448.546,165.725 
 449.588,166.938 450.629,168.151 451.671,169.364 452.712,170.576 453.753,171.787 454.795,172.999 455.836,174.209 456.878,175.419 457.919,176.629 458.961,177.837 
 460.002,179.045 461.044,180.253 462.085,181.46 463.127,182.668 464.168,183.875 465.21,185.083 466.251,186.29 467.293,187.495 468.334,188.7 469.376,189.902 
 470.417,191.103 471.459,192.303 472.5,193.504 473.542,194.706 474.583,195.91 475.625,197.116 476.666,198.321 477.707,199.525 478.749,200.725 479.79,201.921 
 480.832,203.112 481.873,204.301 482.915,205.489 483.956,206.678 484.998,207.871 486.039,209.069 487.081,210.271 488.122,211.476 489.164,212.683 490.205,213.891 
 
 "/>
<polyline clip-path="url(#clip0302)" style="stroke:#e26f46; stroke-width:3; stroke-opacity:1; fill:none" points="
 256.003,124.193 256.751,125.08 257.498,125.97 258.245,126.861 258.992,127.758 259.739,128.659 260.487,129.567 261.234,130.48 261.981,131.396 262.728,132.312 
 263.475,133.226 264.223,134.135 264.97,135.039 265.717,135.94 266.464,136.839 267.211,137.738 267.959,138.64 268.706,139.543 269.453,140.449 270.2,141.355 
 270.948,142.262 271.695,143.167 272.442,144.071 273.189,144.974 273.936,145.875 274.684,146.777 275.431,147.678 276.178,148.58 276.925,149.483 277.672,150.385 
 278.42,151.287 279.167,152.19 279.914,153.091 280.661,153.993 281.408,154.894 282.156,155.794 282.903,156.695 283.65,157.595 284.397,158.496 285.145,159.396 
 285.892,160.296 286.639,161.196 287.386,162.096 288.133,162.995 288.881,163.894 289.628,164.793 290.375,165.692 291.122,166.591 291.869,167.489 292.617,168.388 
 293.364,169.286 294.111,170.183 294.858,171.081 295.605,171.979 296.353,172.876 297.1,173.773 297.847,174.67 298.594,175.566 299.342,176.462 300.089,177.359 
 300.836,178.254 301.583,179.15 302.33,180.046 303.078,180.941 303.825,181.836 304.572,182.731 305.319,183.626 306.066,184.52 306.814,185.414 307.561,186.307 
 308.308,187.201 309.055,188.094 309.802,188.987 310.55,189.88 311.297,190.773 312.044,191.666 312.791,192.559 313.539,193.452 314.286,194.343 315.033,195.233 
 315.78,196.122 316.527,197.011 317.275,197.9 318.022,198.791 318.769,199.683 319.516,200.576 320.263,201.468 321.011,202.36 321.758,203.249 322.505,204.135 
 323.252,205.018 323.999,205.898 324.747,206.779 325.494,207.66 326.241,208.545 326.988,209.434 327.736,210.326 328.483,211.22 329.23,212.116 329.977,213.013 
 
 "/>
<polyline clip-path="url(#clip0302)" style="stroke:#3da44d; stroke-width:3; stroke-opacity:1; fill:none" points="
 176.867,167.775 177.437,168.458 178.007,169.142 178.576,169.828 179.146,170.518 179.715,171.212 180.285,171.912 180.855,172.615 181.424,173.321 181.994,174.027 
 1

We find that more durable capital leads to higher steady state investment and slows the rate of convergence for capital (the slopes are roughly the same, which implies that relative to steady state capital investment responds stronger at higher $\delta$, this in addition to the direct effect of depreciation).

# Use the model to simulate

We will use the deterministic steady-state as a starting point.

In [15]:
set_calibration!(model,delta=0.05)
s0 = model.calibration[:states]
[zip(model.symbols[:states], s0)...]

2-element Array{Tuple{Symbol,Float64},1}:
 (:z, 0.0)    
 (:k, 4.19221)

We also get the covariance matrix just in case. This is a one shock model so all we have it the variance of $e_z$.

In [16]:
sigma2_ez = model.exogenous.Sigma
sigma2_ez

1×1 Array{Float64,2}:
 0.000256

## Impulse response functions

Let us plot the response of consumption and investment to a shock on productivity (to innovation `e_z`)

In [17]:
dr_global = time_iteration(model).dr
irf = response(model, dr_global, :e_z, 0.01)

------------------------------------------------------------------
It    ϵₙ              ηₙ=|xₙ-xₙ₋₁|    λₙ=ηₙ/ηₙ₋₁      Time            Newton steps
------------------------------------------------------------------
1     9.24e-01        1.28e-01        NaN             2.61e-02        7    
2     1.62e-01        4.14e-02        3.25e-01        1.62e-02        5    
3     8.99e-02        2.86e-02        6.90e-01        2.90e-02        5    
4     5.62e-02        2.03e-02        7.09e-01        1.99e-02        5    
5     3.79e-02        1.47e-02        7.27e-01        1.41e-02        4    
6     2.68e-02        1.10e-02        7.43e-01        1.39e-02        4    
7     1.96e-02        8.30e-03        7.57e-01        2.38e-02        4    
8     1.47e-02        6.39e-03        7.70e-01        2.72e-02        4    
9     1.13e-02        4.99e-03        7.81e-01        1.59e-02        4    
10    8.80e-03        3.95e-03        7.91e-01        1.86e-02        4    
11    6.96e-03        3

2-dimensional AxisArray{Float64,2,...} with axes:
    :V, Symbol[:e_z, :z, :k, :n, :i, :w, :rk, :y, :c]
    :T, [1, 2, 3, 4, 5, 6, 7, 8, 9, 10  …  31, 32, 33, 34, 35, 36, 37, 38, 39, 40]
And data, a 9×40 Array{Float64,2}:
 0.0        0.01       0.0        …  0.0         0.0         0.0       
 0.0        0.01       0.008         3.24519e-6  2.59615e-6  2.07692e-6
 4.19221    4.19238    4.20096       4.20105     4.20069     4.20035   
 0.330089   0.330999   0.330549      0.329813    0.329824    0.329834  
 0.209787   0.218194   0.216413      0.209692    0.209695    0.209699  
 1.55       1.56418    1.56281    …  1.55152     1.55145     1.55139   
 0.0601118  0.0608264  0.0605666     0.0599936   0.0599984   0.0600029 
 0.76364    0.77275    0.771023      0.763745    0.76374     0.763736  
 0.553853   0.554556   0.55461       0.554053    0.554045    0.554037  

The easiest way to plot IRF is simply using the stored `irf` values.

In [19]:
### Method for Pyplot
#figure(figsize=(8,4))
#subplot(221)
#plot(irf[:z])
#title("Productivity")
#subplot(222)
#plot(irf[:i])
#title("Investment")
#subplot(223)
#plot(irf[:n])
#title("Labour")
#subplot(224)
#plot(irf[:c])
#title("Consumption")

#tight_layout()

###

### Method for Plots.gr
figure_3 = Plots.plot(Plots.plot(irf[:z], title = "Productivity", line = 2, legend=:none),
    Plots.plot(irf[:i], title = "Investment", line = 2, legend=:none),
    Plots.plot(irf[:n], title = "Labour", line = 2, legend=:none),
    Plots.plot(irf[:c], title = "Consumption", line =2, legend=:none))



<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10 
 
 
 20 
 
 
 30 
 
 
 40 
 
 
 0.000 
 
 
 0.002 
 
 
 0.004 
 
 
 0.006 
 
 
 0.008 
 
 
 0.010 
 
 
 Productivity 
 
<polyline clip-path="url(#clip0502)" style="stroke:#009af9; stroke-width:2; stroke-opacity:1; fill:none" points="
 58.5806,175.813 64.3506,31.4961 70.1206,60.3595 75.8906,83.4503 81.6605,101.923 87.4305,116.701 93.2005,128.524 98.9705,137.982 104.74,145.548 110.51,151.601 
 116.28,156.443 122.05,160.317 127.82,163.417 133.59,165.896 139.36,167.88 145.13,169.466 150.9,170.736 156.67,171.751 162.44,172.564 168.21,173.214 
 173.98,173.734 179.75,174.15 185.52,174.482 191.29,174.749 197.06,174.962 202.83,175.132 208.6,175.268 214.37,175.377 220.14,175.465 225.91,175.534 
 231.68,175.59 237.45,175.635 243.22,175.671 248.99,175.699 254.76,175.722 260.53,175.74 266.3,175.755 272.07,175.767 277.84,175.776 283.61,175.783 
 
 "/>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10 
 
 
 20 
 
 
 30 
 
 
 40 
 
 
 0.210 
 
 
 0.212 
 
 
 0.214 
 
 
 0.216 
 
 
 0.218 
 
 
 Investment 
 
<polyline clip-path="url(#clip0503)" style="stroke:#009af9; stroke-width:2; stroke-opacity:1; fill:none" points="
 348.696,173.613 354.635,31.4961 360.574,61.599 366.513,85.5677 372.452,104.642 378.391,119.81 384.33,131.861 390.269,141.426 396.208,149.008 402.147,155.008 
 408.086,159.748 414.025,163.483 419.963,166.419 425.902,168.718 431.841,170.511 437.78,171.902 443.719,172.974 449.658,173.793 455.597,174.413 461.536,174.876 
 467.475,175.214 473.414,175.456 479.353,175.621 485.292,175.727 491.231,175.788 497.17,175.813 503.109,175.812 509.048,175.791 514.987,175.755 520.926,175.709 
 526.864,175.655 532.803,175.596 538.742,175.535 544.681,175.471 550.62,175.408 556.559,175.345 562.498,175.283 568.437,175.222 574.376,175.164 580.315,175.108 
 
 "/>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10 
 
 
 20 
 
 
 30 
 
 
 40 
 
 
 0.32950 
 
 
 0.32975 
 
 
 0.33000 
 
 
 0.33025 
 
 
 0.33050 
 
 
 0.33075 
 
 
 Labour 
 
<polyline clip-path="url(#clip0504)" style="stroke:#009af9; stroke-width:2; stroke-opacity:1; fill:none" points="
 58.5806,310.698 64.3506,231.496 70.1206,270.641 75.8906,300.613 81.6605,323.338 87.4305,340.347 93.2005,352.858 98.9705,361.839 104.74,368.059 110.51,372.129 
 116.28,374.535 122.05,375.662 127.82,375.813 133.59,375.23 139.36,374.102 145.13,372.578 150.9,370.775 156.67,368.784 162.44,366.676 168.21,364.507 
 173.98,362.316 179.75,360.138 185.52,357.994 191.29,355.903 197.06,353.877 202.83,351.924 208.6,350.05 214.37,348.258 220.14,346.55 225.91,344.925 
 231.68,343.383 237.45,341.922 243.22,340.539 248.99,339.232 254.76,337.998 260.53,336.834 266.3,335.737 272.07,334.703 277.84,333.73 283.61,332.813 
 
 "/>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10 
 
 
 20 
 
 
 30 
 
 
 40 
 
 
 0.5540 
 
 
 0.5542 
 
 
 0.5544 
 
 
 0.5546 
 
 
 Consumption 
 
<polyline clip-path="url(#clip0505)" style="stroke:#009af9; stroke-width:2; stroke-opacity:1; fill:none" points="
 355.286,375.813 361.056,249.891 366.826,240.289 372.596,234.596 378.366,231.904 384.136,231.496 389.906,232.807 395.676,235.393 401.446,238.904 407.216,243.068 
 412.986,247.67 418.756,252.547 424.526,257.571 430.296,262.643 436.066,267.691 441.835,272.658 447.605,277.504 453.375,282.201 459.145,286.728 464.915,291.072 
 470.685,295.226 476.455,299.187 482.225,302.954 487.995,306.53 493.765,309.918 499.535,313.125 505.305,316.155 511.075,319.016 516.845,321.716 522.615,324.26 
 528.385,326.658 534.155,328.916 539.925,331.041 545.695,333.041 551.465,334.922 557.235,336.69 563.005,338.354 568.775,339.917 574.545,341.386 580.315,342.767 
 
 "/>

Note that the plotting is made using the wonderful [matplotlib](http://matplotlib.org/users/pyplot_tutorial.html) library. Read the online [tutorials](http://matplotlib.org/users/beginner.html) to learn how to customize the plots to your needs (e.g., using [latex](http://matplotlib.org/users/usetex.html) in annotations). If instead you would like to produce charts in Matlab, you can easily export the impulse response functions, or any other matrix, to a `.mat` file.

Actually the result of irf is The result is a superconvenient AxisArrays object. In case there is any disagreement about the meanging of signifier "superconvenient", the result can be converted to a DataFrame with the following function:

In [20]:
using AxisArrays
using DataFrames
function to_DataFrame(x::AxisArray{Float64,2})
    _axes = Dict(zip(AxisArrays.axisnames(irf), AxisArrays.axisvalues(irf)))
    colnames = _axes[:V]
    linenames = _axes[:T]
    dd = Dict(v=>irf[Axis{:V}(v)].data for v in colnames)
    return DataFrame(dd)
end

INFO: Recompiling stale cache file /Users/dongxiaozhang/.julia/lib/v0.6/DataArrays.ji for module DataArrays.
INFO: Recompiling stale cache file /Users/dongxiaozhang/.julia/lib/v0.6/DataFrames.ji for module DataFrames.


to_DataFrame (generic function with 1 method)

In [21]:
irf_df = to_DataFrame(irf)
irf_df

,c,e_z,i,k,n,rk,w,y,z
1,0.5538525771807647,0.0,0.20978697257938234,4.1922053700055555,0.3300885083352928,0.060111809698987766,1.5500039699037123,0.763639549760147,0.0
2,0.5545564097035605,0.01,0.21819357027766456,4.19238207408466,0.3309991122281072,0.0608263962796094,1.5641808919131475,0.7727499799812251,0.01
3,0.5546100788278068,0.0,0.21641290352835524,4.200956540658091,0.330549053923697,0.060566583280500956,1.5628100944372258,0.7710229823561621,0.008
4,0.5546418996169873,0.0,0.2149950919055362,4.207321617153542,0.33020445699822465,0.06036624492098199,1.5616287829902402,0.7696369915225235,0.0064
5,0.5546569477683088,0.0,0.2138668218259883,4.211950628201401,0.32994318277172413,0.060212682045234826,1.5606048329370314,0.768523769594297,0.00512
6,0.5546592290515274,0.0,0.21296959379239877,4.2152199186173185,0.32974762666369456,0.0600959182271062,1.5597119424606811,0.7676288228439262,0.004096000000000001
7,0.5546519011322408,0.0,0.21225671655716433,4.217428516478851,0.32960378875625596,0.060008093284483505,1.5589286027045064,0.7669086176894052,0.0032768000000000007
8,0.5546374488353698,0.0,0.21169091588507033,4.218813807212073,0.3295005376190765,0.05994300102209582,1.5582372279958583,0.7663283647204401,0.002621440000000001
9,0.554617823060324,0.0,0.21124243033564363,4.219564032736539,0.3294290258715221,0.05989573369663082,1.557623431686977,0.7658602533959676,0.002097152000000001
10,0.5545945509290533,0.0,0.21088749445555272,4.219828261435356,0.3293822258221523,0.059862406554667735,1.5570754284859567,0.765482045384606,0.001677721600000001


## Stochastic simulations

Now we run 1000 random simulations the result is an AxisArrays indexed by:
- $T$ the number of dates
- $N$ the number of simulations
- $V$ is the number of variables
(the actual ordering of the dimensions is irrelevent if one uses the AxisArrays indexing routines)

In [22]:
sim = simulate(model, dr_global, N=1000, T=1000 )

3-dimensional AxisArray{Float64,3,...} with axes:
    :N, [1, 2, 3, 4, 5, 6, 7, 8, 9, 10  …  991, 992, 993, 994, 995, 996, 997, 998, 999, 1000]
    :V, Symbol[:e_z, :z, :k, :n, :i, :w, :rk, :y, :c]
    :T, [1, 2, 3, 4, 5, 6, 7, 8, 9, 10  …  991, 992, 993, 994, 995, 996, 997, 998, 999, 1000]
And data, a 1000×9×1000 Array{Float64,3}:
[:, :, 1] =
 0.0  0.0  9.35498  0.185209  0.151483  2.4445  0.0238368  0.675737  0.524254
 0.0  0.0  9.35498  0.185209  0.151483  2.4445  0.0238368  0.675737  0.524254
 0.0  0.0  9.35498  0.185209  0.151483  2.4445  0.0238368  0.675737  0.524254
 0.0  0.0  9.35498  0.185209  0.151483  2.4445  0.0238368  0.675737  0.524254
 0.0  0.0  9.35498  0.185209  0.151483  2.4445  0.0238368  0.675737  0.524254
 0.0  0.0  9.35498  0.185209  0.151483  2.4445  0.0238368  0.675737  0.524254
 0.0  0.0  9.35498  0.185209  0.151483  2.4445  0.0238368  0.675737  0.524254
 0.0  0.0  9.35498  0.185209  0.151483  2.4445  0.0238368  0.675737  0.524254
 0.0  0.0  9.35498  0.185209  

In [23]:
# actual ordering of the data: (N,V,T)
size(sim.data)

(1000, 9, 1000)

We plot the responses of consumption, investment and labour to the stochastic path of productivity.

In [29]:
### Method for Pyplot
#figure(figsize=(8,4))
#for i in 500:600
#    subplot(221)
#    plot(sim[Axis{:N}(i),Axis{:V}(:z)], color=:red, alpha=0.1)
#    subplot(222)
#    plot(sim[Axis{:N}(i),Axis{:V}(:i)], color=:red, alpha=0.1)
#    subplot(223)
#    plot(sim[Axis{:N}(i),Axis{:V}(:n)], color=:red, alpha=0.1)
#    subplot(224)
#    plot(sim[Axis{:N}(i),Axis{:V}(:c)], color=:red, alpha=0.1)
#end
#subplot(221)
#title("Productivity")
#subplot(222)
#title("Investment")
#subplot(223)
#title("Labour")
#subplot(224)
#title("Consumption")

#tight_layout()

###

### Method for Plots
#Method_1
#figure_4=Plots.plot(layout = 4)
#for i in 500:600
#    y = [sim[Axis{:N}(i),Axis{:V}(:z)], sim[Axis{:N}(i),Axis{:V}(:i)], sim[Axis{:N}(i),Axis{:V}(:n)], sim[Axis{:N}(i),Axis{:V}(:c)]]
#    Plots.plot!(y, layout = 4, color=:red, alpha=0.1, legend=:none)
#end
#figure_4

#Method_2
p1 = Plots.plot(title = "Productivity")
for i in 500:600
    Plots.plot!(sim[Axis{:N}(i),Axis{:V}(:z)], color=:red, alpha=0.1, xlim = [-10, 1010], ylim = [-0.12, 0.12], legend=:none)
end

p2 = Plots.plot(title = "Investment")
for i in 500:600
    Plots.plot!(sim[Axis{:N}(i),Axis{:V}(:i)], color=:red, alpha=0.1, xlim = [-10, 1010], ylim = [0.12, 0.32], legend=:none)
end

p3 = Plots.plot(title = "Labour")
for i in 500:600
    Plots.plot!(sim[Axis{:N}(i),Axis{:V}(:n)], color=:red, alpha=0.1, xlim = [-10, 1010], ylim = [0.31, 0.355], legend=:none)
end

p4 = Plots.plot(title = "Consumption")
for i in 500:600
    Plots.plot!(sim[Axis{:N}(i),Axis{:V}(:c)], color=:red, alpha=0.1, xlim = [-10, 1010], ylim = [0.53, 0.575], legend=:none)
end

figure_4 = Plots.plot(p1, p2, p3, p4)


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.


It's easy to compare the simulated result with the deterministic steady state value implied from the theoretical model:

In [30]:
mean(sim[Axis{:N}(600),Axis{:V}(:n)])

0.24567127712727843

In [31]:
mean(sim[Axis{:N}(600),Axis{:V}(:i)])

0.17795795344021775

In [32]:
mean(sim[Axis{:N}(600),Axis{:V}(:k)])

7.208851015038012

In [33]:
#set_calibration!(model,delta=0.05)
model.calibration[:controls]

2-element Array{Float64,1}:
 0.33    
 0.233874

In [34]:
model.calibration[:states]

2-element Array{Float64,1}:
 0.0    
 9.35498